<a href="https://colab.research.google.com/github/Nima-Nilchian/stress-detection/blob/master/Stress_type_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import os
import string
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.model_selection import KFold
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

import transformers
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
train = '/content/drive/MyDrive/kaggle/dreaddit-train.csv'
test = '/content/drive/MyDrive/kaggle/dreaddit-test.csv'

In [ ]:
url = train
train_df = pd.read_csv(url,error_bad_lines=False)
train_df = train_df[['text', 'subreddit']]
print('Training data...')
print (train_df.head())

url = test
test_df = pd.read_csv(url, error_bad_lines=False)
test_df = test_df[['text', 'subreddit']]
print('\nTesting data...')
print (test_df.head())

<ipython-input-5-bb369e34f6a4>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train_df = pd.read_csv(url,error_bad_lines=False)


Training data...
                                                text         subreddit
0  He said he had not felt that way before, sugge...              ptsd
1  Hey there r/assistance, Not sure if this is th...        assistance
2  My mom then hit me with the newspaper and it s...              ptsd
3  until i met my new boyfriend, he is amazing, h...     relationships
4  October is Domestic Violence Awareness Month a...  survivorsofabuse


<ipython-input-5-bb369e34f6a4>:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test_df = pd.read_csv(url, error_bad_lines=False)



Testing data...
                                                text      subreddit
0  Its like that, if you want or not.“ ME: I have...  relationships
1  I man the front desk and my title is HR Custom...        anxiety
2  We'd be saving so much money with this new hou...           ptsd
3  My ex used to shoot back with "Do you want me ...           ptsd
4  I haven’t said anything to him yet because I’m...  relationships


**Data Preprocessing**

In [ ]:
train_df["text"] = train_df["text"].str.lower()
test_df["text"] = test_df["text"].str.lower()

In [ ]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

train_df["text"] = train_df["text"].apply(lambda text: remove_punctuation(text))
test_df["text"] = test_df["text"].apply(lambda text: remove_punctuation(text))

In [ ]:
train_df.head()

,text,subreddit
0,he said he had not felt that way before sugget...,ptsd
1,hey there rassistance not sure if this is the ...,assistance
2,my mom then hit me with the newspaper and it s...,ptsd
3,until i met my new boyfriend he is amazing he ...,relationships
4,october is domestic violence awareness month a...,survivorsofabuse


In [ ]:
unique_subreddits = train_df['subreddit'].unique()

In [ ]:
my_dict = {
    'ptsd': 'p',
    'assistance': 'as',
    'relationships': 'r',
    'survivorsofabuse': 'su',
    'domesticviolence':'d',
    'anxiety': 'an',
    'homeless': 'h',
    'stress': 'st',
    'almosthomeless': 'al',
    'food_pantry': 'f'
}

def update_cat(x):
    return my_dict[x]

train_df['subreddit'] = train_df['subreddit'].apply(lambda x: update_cat(x))
test_df['subreddit'] = test_df['subreddit'].apply(lambda x: update_cat(x))

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

train_df['ENCODE_CAT'] = train_df['subreddit'].apply(lambda x: encode_cat(x))
test_df['ENCODE_CAT'] = test_df['subreddit'].apply(lambda x: encode_cat(x))



In [ ]:
train_df.head()

,text,subreddit,ENCODE_CAT
0,he said he had not felt that way before sugget...,p,0
1,hey there rassistance not sure if this is the ...,as,1
2,my mom then hit me with the newspaper and it s...,p,0
3,until i met my new boyfriend he is amazing he ...,r,2
4,october is domestic violence awareness month a...,su,3


In [ ]:
test_df.head()

,text,subreddit,ENCODE_CAT
0,its like that if you want or not“ me i have no...,r,2
1,i man the front desk and my title is hr custom...,an,5
2,wed be saving so much money with this new hous...,p,0
3,my ex used to shoot back with do you want me t...,p,0
4,i haven’t said anything to him yet because i’m...,r,2


In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 2
LEARNING_RATE = 1e-05
NUM_FOLDS = 5
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
kf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        title = str(self.data.subreddit[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))
training_set = Triage(train_df, tokenizer, MAX_LEN)
testing_set = Triage(test_df, tokenizer, MAX_LEN)

TRAIN Dataset: (2838, 3)
TEST Dataset: (715, 3)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 10)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype=torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 2.2565698623657227
Training Accuracy per 5000 steps: 18.75
The Total Accuracy for Epoch 0: 57.71670190274841
Training Loss Epoch: 1.354092891966359
Training Accuracy Epoch: 57.71670190274841
Training Loss per 5000 steps: 0.5149041414260864
Training Accuracy per 5000 steps: 93.75
The Total Accuracy for Epoch 1: 95.59548978153629
Training Loss Epoch: 0.3152059349702315
Training Accuracy Epoch: 95.59548978153629


In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0

    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)

            # Forward pass
            outputs = model(ids, mask)

            # Calculate loss
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()

            # Calculate accuracy
            _, predictions = torch.max(outputs, 1)
            n_correct += torch.sum(predictions == targets).item()

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

    # Calculate accuracy and print validation metrics
    accuracy = (n_correct * 100) / nb_tr_examples
    epoch_loss = tr_loss / nb_tr_steps
    print(f"Validation Loss: {epoch_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.2f}%")

    return accuracy


In [ ]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

This is the validation section to print the accuracy and see how it performs
Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch
Validation Loss: 0.0941
Validation Accuracy: 99.16%
Accuracy on test data = 99.16%
